In [1]:
import os
import git
import json
import sys
sys.path.append("./lib")
import Git
from tqdm import tqdm
import re
from Figure import *

In [2]:
repo_path = "/Users/jinanjiang/Documents/LinuxCommits/linux"
repo = Git.init(repo_path)

In [3]:
with open("./all_commits_by_date/" + "all_dates" + '.json', "r") as f:
    all_dates = json.load(f)
    

In [26]:
file_to_mod_lines_dict = Git.get_modified_lines('6890381720b27a41f2d9e68cce241336342ea3b7', 1, 1)

In [27]:
file_to_stats_dict, overall_counts = Git.get_num_mod_lines(file_to_mod_lines_dict, 1)

In [16]:
def cal_entropy(file_len_lst):
    entropy = 0.0
    total_lines = sum(file_len_lst)
    for l in file_len_lst:
        if l == 0:
            continue
        pi = l / total_lines
        entropy += pi * math.log2(pi)
    return abs((-1) * entropy)

In [18]:
import math

In [28]:
cal_entropy([file_to_stats_dict[k][2] for k in file_to_stats_dict.keys()])

0.0

In [29]:
file_to_stats_dict

{'drivers/gpio/gpio-ws16c48.c': (7, 3, 10)}

In [31]:
p = os.system('pwd')

/Users/jinanjiang/Documents/collecting_data


In [32]:
p

0

In [6]:
def get_info_from_commits(commits):
    from tqdm import tqdm
    com_dict = {}
    for c in tqdm(commits):
        com_dict[c] = Git.get_commit_info(c)
    return com_dict

In [7]:
for date in all_dates[3:4]:
    print(date)
    with open("./all_commits_by_date/" + date + '.json', "r") as f:
        commits = json.load(f)
        
    c = Git.get_info_from_commits(commits)
    
    with open("./all_commits_by_date/" + date + '_info.json', "r") as f:
         ref = json.load(f)
    

2022-09-03


In [25]:
c

{'7726d4c3e60bfe206738894267414a5f10510f1a': {'num_adds_if': 2,
  'num_dels_if': 3,
  'num_mod_total_if': 5,
  'num_adds_loop': 1,
  'num_dels_loop': 2,
  'num_mod_total_loop': 3,
  'file_hash': '7726d4c3e60bfe206738894267414a5f10510f1a',
  'num_adds': 116,
  'num_dels': 106,
  'num_mod_lns_total': 222,
  'mod_files_lst': ('MAINTAINERS',
   'drivers/gpio/gpio-104-dio-48e.c',
   'drivers/gpio/gpio-104-idi-48.c',
   'drivers/gpio/gpio-104-idio-16.c',
   'drivers/gpio/gpio-mockup.c',
   'drivers/gpio/gpio-pca953x.c',
   'drivers/gpio/gpio-pxa.c',
   'drivers/gpio/gpio-realtek-otto.c',
   'drivers/gpio/gpio-ws16c48.c'),
  'entropy': 1.8330608066875809},
 '6890381720b27a41f2d9e68cce241336342ea3b7': {'num_adds_if': 0,
  'num_dels_if': 0,
  'num_mod_total_if': 0,
  'num_adds_loop': 0,
  'num_dels_loop': 0,
  'num_mod_total_loop': 0,
  'file_hash': '6890381720b27a41f2d9e68cce241336342ea3b7',
  'num_adds': 7,
  'num_dels': 3,
  'num_mod_lns_total': 10,
  'mod_files_lst': ('drivers/gpio/gpio-ws1

In [9]:
ref

{'7726d4c3e60bfe206738894267414a5f10510f1a': {'num_adds_if': 2,
  'num_dels_if': 3,
  'num_mod_total_if': 5,
  'num_adds_loop': 4,
  'num_dels_loop': 4,
  'num_mod_total_loop': 8,
  'file_hash': '7726d4c3e60bfe206738894267414a5f10510f1a',
  'num_adds': 135,
  'num_dels': 109,
  'num_mod_lns_total': 244,
  'num_mod_files': 9,
  'is_merge': True,
  'parents': ['65eea2c060ae', '6890381720b2'],
  'mod_files_lst': ['MAINTAINERS',
   'drivers/gpio/gpio-104-dio-48e.c',
   'drivers/gpio/gpio-104-idi-48.c',
   'drivers/gpio/gpio-104-idio-16.c',
   'drivers/gpio/gpio-mockup.c',
   'drivers/gpio/gpio-pca953x.c',
   'drivers/gpio/gpio-pxa.c',
   'drivers/gpio/gpio-realtek-otto.c',
   'drivers/gpio/gpio-ws16c48.c'],
  'entropy': 1.817802455209759,
  'ppl_signed_off': [],
  'ppl_acked': [],
  'ppl_cc': [],
  'ppl_reviewed': [],
  'ppl_tested': [],
  'ppl_reported': [],
  'ppl_suggested': [],
  'ppl_co_developed': [],
  'fixes_lst': [],
  'is_fix': False,
  'mod_dirs': ['drivers/gpio', '/', '/'],
  '

In [ ]:
cal_entropy([file_to_stats_dict[k][2] for k in file_to_stats_dict.keys()])

In [13]:
def strip_comments(diff_text):

    # one way to simplify things is to preprocess. This also provides reference for other stuff. Also line counts for totals.
    in_file = False
    in_comment = False
    new_file = []
    for cur_l in diff_text:
        #print(cur_l)
        if not in_file:
            if cur_l[:2] == '@@':
                in_file = True
            new_file.append(cur_l)
            continue
        else:
            if cur_l[:4] == 'diff':
                if in_comment:
                    with open("errors.txt", 'a') as f:
                        json.dump('**********' + "       " + commit + '**********', indent=1)
                in_file = False
                new_file.append(cur_l)
                continue

        new_cur_ln = cur_l[0]
        cur_l = cur_l[1:]

        while True:
            # consider //
            #print(cur_l)
            if in_comment:
                dex = cur_l.find('*/')
                if dex == -1:
                    break
                else:
                    in_comment = False
                cur_l = cur_l[dex + 2:]
                if len(cur_l) == 0:
                    break

            else:
                dex = cur_l.find('/*')
                double_bar_dex = cur_l.find('//')

                if dex == -1:
                    if double_bar_dex == -1:
                        new_cur_ln += cur_l
                        break
                    else:
                        cur_l = cur_l[:double_bar_dex]
                        new_cur_ln += cur_l
                        break
                else:
                    # there is /*
                    if double_bar_dex != -1:
                        if double_bar_dex < dex:
                            cur_l = cur_l[:double_bar_dex]
                            new_cur_ln += cur_l
                            break
                        else:
                            in_comment = True
                    else:
                        in_comment = True


                if dex > 0:
                    new_cur_ln += cur_l[:dex]
                cur_l = cur_l[dex:]
                #print(cur_l)
                if len(cur_l) == 0:
                    break


        new_file.append(new_cur_ln)
    return new_file





In [ ]:
diff_text = [i for i in diff_text if i.startswith("diff") or i.startswith("+") or i.startswith("-")]
    
    if filter_empty_line:
        diff_text = [i for i in diff_text if len(i.strip()) != 0]
        
    file_to_mod_lines_dict = {}
    file_nm = None
    
    cur_line = 0
    num_total_lines = len(diff_text)
    while cur_line < num_total_lines:
        cur_text = diff_text[cur_line]
        if cur_text[:4] == 'diff':
            file_nm = cur_text.split()[-1]

            if file_nm[:2] == "a/":
                raise Exception("wrong wrong wrong")
            if file_nm[:2] == "b/":
                file_nm = file_nm[2:]

            
            file_to_mod_lines_dict[file_nm] = []

            if cur_line + 1 == num_total_lines:
                # no more lines to consume, just exit
                break
            next_line_st = diff_text[cur_line + 1][0]
            if next_line_st == "+" or next_line_st == "-":
                cur_line += 3 # skip 2 lines
            else:
                next_line_st = diff_text[cur_line + 1][:4]
                assert next_line_st == "diff"
                cur_line += 1
            continue
        else:
            first_char = cur_text[0]
            assert first_char == '+' or first_char == '-'
            file_to_mod_lines_dict[file_nm].append(cur_text)
            cur_line += 1
    return file_to_mod_lines_dict